In this tutorial, we will discuss how to classify images into pictures of cats or pictures of dogs. We'll build an image classifier using tf.keras.Sequential model and load data using tf.keras.preprocessing.image.ImageDataGenerator.

# Specific concepts that will be covered:
In the process, we will build practical experience and develop intuition around the following concepts

Building data input pipelines using the tf.keras.preprocessing.image.ImageDataGenerator class — How can we efficiently work with data on disk to interface with our model?
Overfitting - what is it, how to identify it?

Let's start by importing required packages:

os — to read files and directory structure

numpy — for some matrix math outside of TensorFlow

matplotlib.pyplot — to plot the graph and display images in our training and validation data

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import math

# Data Loading
To build our image classifier, we begin by downloading the dataset. The dataset we are using is a filtered version of Dogs vs. Cats dataset from Kaggle (ultimately, this dataset is provided by Microsoft Research).

In previous Colabs, we've used TensorFlow Datasets, which is a very easy and convenient way to use datasets. In this Colab however, we will make use of the class tf.keras.preprocessing.image.ImageDataGenerator which will read data from disk. We therefore need to directly download Dogs vs. Cats from a URL and unzip it to the Colab filesystem.

In [2]:
#_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path = os.getcwd()
print(path)
dir_path = os.path.join(path, "cats_and_dogs_filtered")
print(dir_path)

C:\Users\Mustafa Hasnain\Desktop\Udacity Introduction to TensorFlow- Deep Learning
C:\Users\Mustafa Hasnain\Desktop\Udacity Introduction to TensorFlow- Deep Learning\cats_and_dogs_filtered


In [3]:
train_path = os.path.join(dir_path, "train")
validation_path = os.path.join(dir_path, "validation")
train_cats_path = os.path.join(train_path, "cats")
train_dogs_path = os.path.join(train_path, "dogs")
validation_cats_path = os.path.join(validation_path, "cats")
validation_dogs_path = os.path.join(validation_path, "dogs")
print("Directory Containing tain dataset for cats and dogs: \n",train_cats_path, "\n",train_dogs_path)
print("Directory Containing Validation dataset for cats and dogs: \n",validation_cats_path, "\n",validation_dogs_path)

Directory Containing tain dataset for cats and dogs: 
 C:\Users\Mustafa Hasnain\Desktop\Udacity Introduction to TensorFlow- Deep Learning\cats_and_dogs_filtered\train\cats 
 C:\Users\Mustafa Hasnain\Desktop\Udacity Introduction to TensorFlow- Deep Learning\cats_and_dogs_filtered\train\dogs
Directory Containing Validation dataset for cats and dogs: 
 C:\Users\Mustafa Hasnain\Desktop\Udacity Introduction to TensorFlow- Deep Learning\cats_and_dogs_filtered\validation\cats 
 C:\Users\Mustafa Hasnain\Desktop\Udacity Introduction to TensorFlow- Deep Learning\cats_and_dogs_filtered\validation\dogs


In [4]:
#Or another method to deownload and get path of dataset is: 
#_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
#zip_dir = tf.keras.utils.get_file('cats_and_dogs_filterted.zip', origin=_URL, extract=True)
#zip_dir_base = os.path.dirname(zip_dir)

# Understanding Our data

In [5]:
num_cats_tr = os.listdir(train_cats_path) #os.Listdir returns the files in the path given/directory
num_dogs_tr = os.listdir(train_dogs_path)
num_cats_val = os.listdir(validation_cats_path)
num_dogs_val = os.listdir(validation_dogs_path)
print("The No of files in Train-Cats Directory: ",len(num_cats_tr))
print("The No of files in Train-Dogs Directory: ",len(num_dogs_tr))
print("The No of files in Validation-Cats Directory: ",len(num_cats_val))
print("The No of files in Validation-Dogs Directory: ",len(num_cats_val))

The No of files in Train-Cats Directory:  1000
The No of files in Train-Dogs Directory:  1000
The No of files in Validation-Cats Directory:  500
The No of files in Validation-Dogs Directory:  500


In [6]:
total_train = len(num_cats_tr + num_dogs_tr)
total_val = len(num_cats_val+ num_dogs_val)
print("Total Number of Images in Train: ", total_train)
print("Total Number of Images in Validation: ", total_val)

Total Number of Images in Train:  2000
Total Number of Images in Validation:  1000


# Setting Model Parameters
For convenience, we'll set up variables that will be used later while pre-processing our dataset and training our network.

In [7]:
BATCH_SIZE = 100 # Number of training examples to process before updating our models variables
IMG_SHAPE = 150 # Our training data consists of images with width of 150 pixels and height of 150 pixels

# Data Preparation
Images must be formatted into appropriately pre-processed floating point tensors before being fed into the network. The steps involved in preparing these images are:

Read images from the disk
Decode contents of these images and convert it into proper grid format as per their RGB content
Convert them into floating point tensors
Rescale the tensors from values between 0 and 255 to values between 0 and 1, as neural networks prefer to deal with small input values.
Fortunately, all these tasks can be done using the class tf.keras.preprocessing.image.ImageDataGenerator.

We can set this up in a couple of lines of code.

In [8]:
train_img_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_img_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

After defining our generators for training and validation images, flow_from_directory method will load images from the disk, apply rescaling, and resize them using single line of code.

In [9]:
train_data_gen = train_img_generator.flow_from_directory(directory=train_path, batch_size=BATCH_SIZE, shuffle=True,class_mode='binary',target_size = (IMG_SHAPE,IMG_SHAPE))
val_data_gen = validation_img_generator.flow_from_directory(directory=validation_path, batch_size=BATCH_SIZE, shuffle=False,class_mode='binary',target_size = (IMG_SHAPE,IMG_SHAPE))
#target_size = (150,150) px
#In Validation and Test Data Shuffling is mostly likely False

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [10]:
print(train_data_gen.image_shape)
print(val_data_gen.image_shape)

(150, 150, 3)
(150, 150, 3)


# Visualizing Training images
We can visualize our training images by getting a batch of images from the training generator, and then plotting a few of them using matplotlib.

In [11]:
sample_images,_ = next(train_data_gen)

In [1]:
def plot_image(img_list):
    fig, axes = plt.subplots(1,5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(img_list, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()
plot_image(sample_images[:5])

NameError: name 'sample_images' is not defined

# Model Creation
Define the model
The model consists of four convolution blocks with a max pool layer in each of them. Then we have a fully connected layer with 512 units, with a relu activation function. The model will output class probabilities for two classes — dogs and cats — using softmax.



In [13]:
def create_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', input_shape = (150,150,3)))
    model.add(tf.keras.layers.MaxPooling2D(2,2))
    model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(2,2))
    model.add(tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'))
    model.add(tf.keras.layers.MaxPooling2D(2,2))
    model.add(tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'))
    model.add(tf.keras.layers.MaxPooling2D(2,2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units = 512, activation = 'relu'))
    model.add(tf.keras.layers.Dense(units = 2))
    model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    return model

In [14]:
model = create_model()
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

In [ ]:
EPOCHS = 10
#Train model
history = model.fit_generator(train_data_gen, steps_per_epoch = math.ceil(total_train/BATCH_SIZE), epochs=EPOCHS, validation_data=val_data_gen, validation_steps=np.ceil(total_val/BATCH_SIZE))

C:\Users\Mustafa Hasnain\anaconda3\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
20/20 [==============================] - 65s 3s/step - loss: 0.7568 - accuracy: 0.4875 - val_loss: 0.6929 - val_accuracy: 0.5010
Epoch 2/10
13/20 [==================>...........] - ETA: 19s - loss: 0.6928 - accuracy: 0.4915